# 2.12 Principal Components Analysis(PCA)

很简单的机器学习算法，主成分分析(Principal Components Analysis, PCA) 只需要使用基本的线性代数的知识即可。

假设我们有$m$个点的集合${𝒙}^{(1)},...,{𝒙}^{(m)}\inℝ^n$,同时，假设我们想要对这些点进行有损失的压缩的话，我们就需要尽可能少的损失精度。这里的${𝒙}^{(i)}$表示标准基向量$[0,...,0,1,0,...,0]$，在第$i$个位置的值为1,其余位置为0.

一种编码这些点的办法就是使用这些点的低维表示，对于每个${𝒙}^{(i)}\inℝ^n$点可以找到相应的编码向量$𝒄^{(i)}\in\mathbb{R}^l$，如果$l<n$的话，那么相比于原始点，新的编码点就可以需要更少的存储空间。PCA的任务也就是找到一些编码函数，让每个点作为输入：$f(𝒙)=𝒄$以生成编码点，同时一个解码函数能够重构输入：$𝒙≈g(f(𝒙))$。

通过我们的选择解码函数来定义 PCA。特别地，为了让解码简单，我们选择使用矩阵乘来将编码映射到$\mathbb{R}^n$。令$g(𝒄)=𝑫𝒄$，其中$𝑫\in\mathbb{R}^{n\times l}$为解码矩阵。

因为这里计算解码的最优编码是一个很困难的问题，为了保证编码问题简单，PCA 限制矩阵$𝑫$的列向量互相正交(只有在$l=n$时，矩阵$𝑫$才是一个正交矩阵)。

对于目前描述的问题，有许多解决方案，因为如果我们按比例减少所有点的$c_i$，我们可以增加$𝑫_{:,i}$的规模。为了给问题一个唯一的解决方案，我们将$𝑫$的所有列限制为具有单位范数。

为了让基本思路能够变成我们可以实现的算法话，我们的第一件事情需要知道，如何对每个输入点$𝒙$，生成最优的编码点$𝒄^*$。能够实现的办法是**最小化**输入点$𝒙$及其重构点$g(𝒄^*)$之间的距离。我们可以使用范数来测量距离，在 PCA 算法中，我们使用$L^2$范数:

$$
𝒄^*=\operatorname{arg min}_{𝒄}||𝒙-g(𝒄)||_2
$$

可以使用平方二范数以替代二范数，因为两者都是对相同值$𝒄$的最小化。因为$L^2$范数是非负值，而平方计算操作对于非负值来讲是单调递增的。

$$
𝒄^*=\operatorname{arg min}_{𝒄}||𝒙-g(𝒄)||^2_2
$$

故而，函数的最小化简化为

$$
(𝒙-g(𝒄))^T(𝒙-g(𝒄))
$$

通过二范数的定义，我们可以得到

$$
𝒙^T𝒙-𝒙^Tg(𝒄)-g(𝒄)^T𝒙+g(𝒄)^Tg(𝒄)
$$

因为标量$g(𝒄)^T𝒙$等于转置本身，所以有

$$
x^T𝒙-2𝒙^Tg(𝒄)+g(𝒄)^Tg(𝒄)
$$

故而，消除掉第一项(第一项不依赖$𝒄$)，我们就可以再一次改变最小化函数：

$$
𝒄^*=\operatorname{arg min}_{𝒄}-2𝒙^Tg(𝒄)+g(𝒄)^Tg(𝒄)
$$

进而，我们可以使用$g(𝒄)$的定义进行替换，得到：

$$
𝒄^*=\operatorname{arg min}_{𝒄}-2𝒙^T𝑫𝒄+𝒄^T𝑫^T𝑫𝒄
$$

$$
=\operatorname{arg min}_{𝒄}-2𝒙^T𝑫𝒄+𝒄^T𝑰_l𝒄
$$

因为$𝑫$上的正交和单位范数的限制，我们就可以得到：

$$
=\operatorname{arg min}_{𝒄}-2𝒙^T𝑫𝒄+𝒄^T𝒄
$$

我们可以使用向量导数来解决这个优化问题：

$$
∇_{𝒄}(-2𝒙^T𝑫𝒄+𝒄^T𝒄)=0
$$

$$
-2𝑫^T𝒙+2𝒄=0
$$

$$
𝒄=𝑫^T𝒙
$$

这样就会让算法更加高效：我们只需使用矩阵向量运算就可以对$𝒙$进行最佳编码。要对向量进行编码，我们应用编码器函数:

$$
f(𝒙)=𝑫^T𝒙
$$

进一步矩阵乘，我们也可以定义PCA的重构操作：

$$
r(𝒙)=g(f(𝒙))=𝑫𝑫^T𝒙
$$

下一步，我们需要选择编码矩阵$𝑫$。为了达成，我们需要重新审视输入和重构之间的最下化$L^2$距离。因为使用相同的矩阵$𝑫$去编码所有的点，因此我们不能再孤立地考虑这些点。相反，我们必须最小化在所有维度和所有点上计算的误差矩阵的 Frobenius 范数：

$$
𝑫^*=\operatorname{arg min}_{𝑫}\sqrt{∑_{i,j}(x_j^{(i)}-r(𝒙^{(i)}))_j)^2}
$$
且满足：$𝑫^T𝑫=𝑰_l$

为了推导寻找$𝑫^*$的算法，首先考虑$l=1$的情况。在这种情况下，$𝑫$是一个向量$𝒅$，进而问题就变为：

$$
𝒅^*=\operatorname{arg min}_{𝒅}∑_i||𝒙^{(i)}-𝒅𝒅^T𝒙^{(i)}||_2^2
$$
且满足：$||𝒅||_2=1$

上述公式是执行替换的最直接方式，但不是写出方程式最令人愉悦的方式。它将标量值$𝒅^T𝒙^{(i)}$放在向量$𝒅$的右侧。将标量系数写在它们所作用的向量的左侧更为常见。因此，我们通常将这样的公式写为:

$$
𝒅^*=\operatorname{arg min}_{𝒅}∑_i||𝒙^{(i)}-𝒅^T𝒙^{(i)}𝒅||_2^2
$$
且满足：$||𝒅||_2=1$

或者，因为标量是本身的转置，可以得到：

$$
𝒅^*=\operatorname{arg min}_{𝒅}∑_i||𝒙^{(i)}-𝒙^{(i)T}𝒅𝒅||_2^2
$$
且满足：$||𝒅||_2=1$

读者应该努力熟悉这种表面重组。此时，将问题重写为单个示例设计矩阵，而不是单独示例向量的总和，会很有帮助。这将使我们能够使用更紧凑的符号。让$𝑿\inℝ^{m×n}$成为通过堆叠所有描述点的向量定义的矩阵，使得$𝑿_{i,:}=𝒙^{(i)T}$。我们现在可以将问题重写为：

$$
𝒅^*=\operatorname{arg min}_{𝒅}||𝑿-𝑿𝒅𝒅^T||^2_F
$$
且满足：$||𝒅||_2=1$

暂时忽略约束，我们可以将 Frobenius 范数部分简化如下：

$$
\operatorname{arg min}_{𝒅}||𝑿-𝑿𝒅𝒅^T||^2_F
$$

$$
=\operatorname{arg min}_{𝒅}\operatorname{Tr}((𝑿-𝑿𝒅𝒅^T)^T(𝑿-𝑿𝒅𝒅^T))
$$
其中，F范数的定义：$||𝑨||_F=\sqrt{\operatorname{Tr}(𝑨𝑨^T)}$。

$$
=\operatorname{arg min}_{𝒅}\operatorname{Tr}(𝑿^T𝑿-𝑿^T𝑿𝒅𝒅^T-𝒅𝒅^T𝑿^T𝑿+𝒅𝒅^T𝑿^T𝑿𝒅𝒅^T)
$$

$$
=\operatorname{arg min}_{𝒅}\operatorname{Tr}(𝑿^T𝑿)-\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T)-\operatorname{Tr}(𝒅𝒅^T𝑿^T𝑿)+\operatorname{Tr}(𝒅𝒅^T𝑿^T𝑿𝒅𝒅^T)
$$

$$
=\operatorname{arg min}_{𝒅}-\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T)-\operatorname{Tr}(𝒅𝒅^T𝑿^T𝑿)+\operatorname{Tr}(𝒅𝒅^T𝑿^T𝑿𝒅𝒅^T)
$$

$$
=\operatorname{arg min}_{𝒅}-2\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T)+\operatorname{Tr}(𝒅𝒅^T𝑿^T𝑿𝒅𝒅^T)
$$

因为我们可以循环迹内的矩阵顺序：

$$
=\operatorname{arg min}_{𝒅}-2\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T)+\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T𝒅𝒅^T)
$$
因为，
$$
\operatorname{Tr}(∏_{i=1}^n𝑭^{(i)})=\operatorname{Tr}(𝑭^{(i)}∏_{i=1}^{n-1}𝑭^{(i)})
$$

也就得到了：

$$
=\operatorname{arg min}_{𝒅}-2\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T)+\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T)
$$
因为：$||𝒅||_2=𝒅^T𝒅=1$

也就是：

$$
=\operatorname{arg min}_{𝒅}-\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T)
$$

$$
=\operatorname{arg min}_{𝒅}\operatorname{Tr}(𝑿^T𝑿𝒅𝒅^T)
$$

$$
=\operatorname{arg min}_{𝒅}\operatorname{Tr}(𝒅^T𝑿^T𝑿𝒅)
$$

且满足：$𝒅^T𝒅=1$

可以使用特征分解来解决此优化问题。具体而言，最优$𝒅$由对应于最大特征值的$𝑿^T𝑿$的特征向量给出。此推导特定于$l=1$的情况，并且仅恢复第一个主成分。更一般地，当我们希望恢复主成分的基础时，矩阵$𝑫$由对应于最大特征值的$l$个特征向量给出。这可以使用归纳证明来显示。我们建议将此证明写成练习。线性代数是理解深度学习所必需的基础数学学科之一。机器学习中无处不在的另一个关键数学领域是概率论，接下来介绍。